In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set the style for visualizations
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

# 1. Data Cleaning
# Load the dataset
print("Loading the dataset...")
try:
    # Try loading the data from the provided file path
    retail_data = pd.read_csv('./retail_data.csv')
    print(f"Successfully loaded {len(retail_data)} records.")
except:
    # Generate random data if file loading fails
    print("File not found, generating synthetic data...")
    # Generate random data
    num_records = 1000
    categories = ['Electronics', 'Clothing', 'Groceries', 'Home & Garden', 'Health & Beauty']
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2024, 10, 1)

    # Generate random data
    data = {
        'order_id': [f'ORD{str(i).zfill(6)}' for i in range(1, num_records + 1)],
        'customer_id': [f'CUST{np.random.randint(1, 201)}' for _ in range(num_records)],
        'order_date': [start_date + timedelta(days=np.random.randint(0, (end_date - start_date).days)) for _ in range(num_records)],
        'product_id': [f'PROD{np.random.randint(1, 101)}' for _ in range(num_records)],
        'quantity': np.random.randint(1, 10, size=num_records),
        'price': np.round(np.random.uniform(5.0, 500.0, size=num_records), 2),
        'category': [np.random.choice(categories) for _ in range(num_records)]
    }
    
    # Create DataFrame
    retail_data = pd.DataFrame(data)
    # Convert datetime objects to strings
    retail_data['order_date'] = retail_data['order_date'].astype(str)
    print(f"Generated {len(retail_data)} records.")

# Display the first few rows
print("\nFirst 5 rows of the dataset:")
print(retail_data.head())

# Check data types
print("\nData types:")
print(retail_data.dtypes)

# Check for missing values
print("\nMissing values check:")
missing_values = retail_data.isnull().sum()
print(missing_values)

# If there are missing values, handle them
if missing_values.sum() > 0:
    print("\nHandling missing values...")
    # For numerical columns, fill with median
    numerical_cols = retail_data.select_dtypes(include=['float64', 'int64']).columns
    for col in numerical_cols:
        if retail_data[col].isnull().sum() > 0:
            retail_data[col] = retail_data[col].fillna(retail_data[col].median())
    
    # For categorical columns, fill with mode
    categorical_cols = retail_data.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if retail_data[col].isnull().sum() > 0 and col != 'order_date':
            retail_data[col] = retail_data[col].fillna(retail_data[col].mode()[0])
    
    # For order_date, fill with previous date
    if 'order_date' in retail_data.columns and retail_data['order_date'].isnull().sum() > 0:
        retail_data['order_date'] = retail_data['order_date'].fillna(method='ffill')

    print("After handling missing values:")
    print(retail_data.isnull().sum())

# Convert order_date to datetime
print("\nConverting order_date to datetime...")
retail_data['order_date'] = pd.to_datetime(retail_data['order_date'])

# Extract year and month into new columns
retail_data['order_year'] = retail_data['order_date'].dt.year
retail_data['order_month'] = retail_data['order_date'].dt.month
retail_data['order_yearmonth'] = retail_data['order_date'].dt.strftime('%Y-%m')

print("After date conversion and extraction:")
print(retail_data.head())

# 2. Exploratory Data Analysis (EDA)
# Generate summary statistics for numerical columns
print("\nSummary statistics for numerical columns:")
print(retail_data[['quantity', 'price']].describe())

# Distribution of sales by product category
plt.figure(figsize=(12, 6))
category_counts = retail_data['category'].value_counts()
sns.barplot(x=category_counts.index, y=category_counts.values)
plt.title('Distribution of Products by Category')
plt.xlabel('Category')
plt.ylabel('Number of Products')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('category_distribution.png')
plt.close()

# 3. Feature Engineering
# Create a new feature called total_sales which is calculated as quantity * price
print("\nCreating total_sales feature...")
retail_data['total_sales'] = retail_data['quantity'] * retail_data['price']
print("Sample data with total_sales:")
print(retail_data[['quantity', 'price', 'total_sales']].head())

# Create additional features that might be relevant for predicting future sales
# Customer order frequency
customer_order_counts = retail_data.groupby('customer_id')['order_id'].count()
retail_data['customer_order_frequency'] = retail_data['customer_id'].map(customer_order_counts)

# Average price per category
category_avg_price = retail_data.groupby('category')['price'].mean()
retail_data['category_avg_price'] = retail_data['category'].map(category_avg_price)

# Product popularity (number of times ordered)
product_popularity = retail_data.groupby('product_id')['order_id'].count()
retail_data['product_popularity'] = retail_data['product_id'].map(product_popularity)

# Day of week
retail_data['order_day_of_week'] = retail_data['order_date'].dt.dayofweek

# Month-end flag (1 if the order was placed in the last 5 days of the month)
retail_data['is_month_end'] = ((retail_data['order_date'].dt.day >= 25) & 
                              (retail_data['order_date'].dt.day <= 31)).astype(int)

# Season
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

retail_data['season'] = retail_data['order_month'].apply(get_season)

# One-hot encode the season
season_dummies = pd.get_dummies(retail_data['season'], prefix='season')
retail_data = pd.concat([retail_data, season_dummies], axis=1)

# One-hot encode the category
category_dummies = pd.get_dummies(retail_data['category'], prefix='category')
retail_data = pd.concat([retail_data, category_dummies], axis=1)

print("\nAfter feature engineering:")
print(retail_data.columns.tolist())

# Monthly sales trend
monthly_sales = retail_data.groupby('order_yearmonth')['total_sales'].sum().reset_index()
plt.figure(figsize=(15, 6))
sns.lineplot(x='order_yearmonth', y='total_sales', data=monthly_sales, marker='o')
plt.title('Monthly Sales Trend')
plt.xlabel('Year-Month')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.savefig('monthly_sales_trend.png')
plt.close()

# Identify the top 5 products by total sales
top_products = retail_data.groupby('product_id')['total_sales'].sum().reset_index()
top_products = top_products.sort_values('total_sales', ascending=False).head(5)
plt.figure(figsize=(12, 6))
sns.barplot(x='product_id', y='total_sales', data=top_products)
plt.title('Top 5 Products by Total Sales')
plt.xlabel('Product ID')
plt.ylabel('Total Sales')
plt.tight_layout()
plt.savefig('top_products.png')
plt.close()

# Sales by category
category_sales = retail_data.groupby('category')['total_sales'].sum().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x='category', y='total_sales', data=category_sales)
plt.title('Total Sales by Product Category')
plt.xlabel('Category')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('category_sales.png')
plt.close()

# 4. Modeling
# Define features to use in the model
features = [
    'quantity', 'price', 'customer_order_frequency', 'category_avg_price',
    'product_popularity', 'order_day_of_week', 'is_month_end',
    'season_Fall', 'season_Spring', 'season_Summer', 'season_Winter',
]

# Add category dummies to features
category_dummy_cols = [col for col in retail_data.columns if col.startswith('category_')]
features.extend(category_dummy_cols)

# Define the target
target = 'total_sales'

# Prepare the data
X = retail_data[features]
y = retail_data[target]

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nTraining and testing data shape:")
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")

# Linear Regression Model
print("\nTraining Linear Regression model...")
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predict on test set
y_pred_lr = lr_model.predict(X_test)

# Evaluate the model
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("\nLinear Regression Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse_lr:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse_lr:.2f}")
print(f"R-squared (R²): {r2_lr:.4f}")

# Random Forest Model
print("\nTraining Random Forest model...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("\nRandom Forest Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse_rf:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse_rf:.2f}")
print(f"R-squared (R²): {r2_rf:.4f}")

# Feature importance (for Random Forest)
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
})
feature_importance = feature_importance.sort_values('Importance', ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance)
plt.title('Top 10 Feature Importance')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()

# Actual vs. Predicted Plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_rf, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.title('Actual vs. Predicted Sales (Random Forest)')
plt.xlabel('Actual Sales')
plt.ylabel('Predicted Sales')
plt.tight_layout()
plt.savefig('actual_vs_predicted.png')
plt.close()

# Correlation matrix of numerical features
correlation_matrix = retail_data[['quantity', 'price', 'total_sales', 
                                 'customer_order_frequency', 'category_avg_price', 
                                 'product_popularity']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.tight_layout()
plt.savefig('correlation_matrix.png')
plt.close()

print("\nAnalysis and modeling completed successfully!")

Loading the dataset...
Successfully loaded 1000 records.

First 5 rows of the dataset:
    order_id customer_id  order_date product_id  quantity   price  \
0  ORD000001     CUST126  2022-06-19      PROD5         4  305.76   
1  ORD000002      CUST44  2023-03-15     PROD66         8   13.85   
2  ORD000003     CUST199  2022-12-03      PROD8         9  372.73   
3  ORD000004     CUST100  2022-11-28     PROD75         2  119.40   
4  ORD000005     CUST114  2024-08-31     PROD51         5  154.89   

          category  
0         Clothing  
1  Health & Beauty  
2    Home & Garden  
3         Clothing  
4        Groceries  

Data types:
order_id        object
customer_id     object
order_date      object
product_id      object
quantity         int64
price          float64
category        object
dtype: object

Missing values check:
order_id       0
customer_id    0
order_date     0
product_id     0
quantity       0
price          0
category       0
dtype: int64

Converting order_date to dat